In [ ]:
import pandas as pd
import random

# Load data
classrooms_df = pd.read_csv('/content/classrooms.csv')  # Update if needed
teachers_df = pd.read_csv('/content/teachers.csv')

# Clean up column names
classrooms_df.columns = classrooms_df.columns.str.strip()
teachers_df.columns = teachers_df.columns.str.strip()

# Auto-detect column names (first two for classroom, first for teachers)
room_col = classrooms_df.columns[0]
class_col = classrooms_df.columns[1]
teacher_col = teachers_df.columns[0]

# 🔸 Step 1: Ask user how many classrooms are needed
requested_classroom_count = int(input("🔢 How many classrooms are needed for the exam? "))

# 🔸 Step 2: Ask unavailable Room Nos and Teachers
unavailable_classrooms_input = input("🚫 Enter unavailable Room Nos (comma-separated) or 0 if none: ")
unavailable_teachers_input = input("🚫 Enter unavailable Teacher Names (comma-separated) or 0 if none: ")

unavailable_rooms = [] if unavailable_classrooms_input.strip() == '0' else [room.strip() for room in unavailable_classrooms_input.split(',')]
unavailable_teachers = [] if unavailable_teachers_input.strip() == '0' else [t.strip() for t in unavailable_teachers_input.split(',')]

# 🔸 Step 3: Filter out unavailable ones
available_classrooms_df = classrooms_df[~classrooms_df[room_col].astype(str).isin(unavailable_rooms)].reset_index(drop=True)
available_teachers_df = teachers_df[~teachers_df[teacher_col].isin(unavailable_teachers)].reset_index(drop=True)

# 🔸 Step 4: Validation
if requested_classroom_count > len(available_classrooms_df):
    print(f"❌ Only {len(available_classrooms_df)} classrooms are available!")
elif len(available_teachers_df) < requested_classroom_count * 2:
    print(f"❌ At least {requested_classroom_count * 2} teachers are needed. Only {len(available_teachers_df)} available.")
else:
    # Select classrooms
    selected_classrooms = available_classrooms_df.sample(n=requested_classroom_count).reset_index(drop=True)

    # Shuffle teachers
    teacher_list = available_teachers_df[teacher_col].tolist()
    random.shuffle(teacher_list)

    fixed_duty = teacher_list[:requested_classroom_count]
    relieving_duty = teacher_list[requested_classroom_count:requested_classroom_count*2]
    spare_teachers = teacher_list[requested_classroom_count*2:]

    # Final schedule
    duty_schedule = pd.DataFrame({
        "Room No": selected_classrooms[room_col],
        "Class": selected_classrooms[class_col],
        "Fixed Duty Teacher": fixed_duty,
        "Relieving Duty Teacher": relieving_duty
    })

    # Output
    print("\n✅ Final Exam Duty Schedule:")
    display(duty_schedule)

    if spare_teachers:
        print("\n🆓 Spare Teachers:\n", ", ".join(spare_teachers))

    # Save
    duty_schedule.to_excel("final_exam_duty_schedule.xlsx", index=False)
    print("\n📁 Saved as 'final_exam_duty_schedule.xlsx'")


🔢 How many classrooms are needed for the exam? 10
🚫 Enter unavailable Room Nos (comma-separated) or 0 if none: r5,R8
🚫 Enter unavailable Teacher Names (comma-separated) or 0 if none: T12,T15

✅ Final Exam Duty Schedule:


,Room No,Class,Fixed Duty Teacher,Relieving Duty Teacher
0,R9,9,T18,T5
1,R15,12B,T20,T26
2,R4,4,T23,T21
3,R13,11B,T7,T14
4,R7,7,T19,T6
5,R11,10B,T30,T9
6,R10,10A,T22,T8
7,R2,2,T29,T10
8,R12,11A,T27,T13
9,R3,3,T4,T2



🆓 Spare Teachers:
 T28, T17, T11, T16, T24, T3, T25

📁 Saved as 'final_exam_duty_schedule.xlsx'
